In [ ]:
!nvidia-smi

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import time
import sys
import matplotlib.pyplot as plt
import matplotlib.lines as lns
import numpy as np
import math

from google.colab import files
from functools import reduce

torch.manual_seed(1)



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])

dnload = True
batch_sz = 128

# trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        # download=dnload, transform=transform)
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=dnload, transform=transform)
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=dnload, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_sz,
                                          shuffle=True, num_workers=2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data


In [ ]:
for randseed in range(20):
      torch.manual_seed(10+randseed)

      
      net = models.resnet18()
      net.to(device)

      criterion = nn.CrossEntropyLoss()

      # parameters for ADAM related methods
      beta2=0.999
      beta1=0.9




      learning_rate = 0.1
      optimizer = optim.SGD(net.parameters(),  lr=learning_rate) # learning_rate = 0.1
      # optimizer = optim.SGD(net.parameters(),  lr=learning_rate, momentum=0.9, nesterov=True) # learning_rate = 0.01
      # optimizer = optim.SGD(net.parameters(),  lr=learning_rate, momentum=0.9, nesterov=False) # learning_rate = 0.01
      # optimizer = optim.AdamW(net.parameters(),betas=(0.9, 0.999), lr=learning_rate,amsgrad=True) # learning_rate = 0.001
      # optimizer = optim.Adam(net.parameters(),betas=(beta1, beta2), lr=learning_rate,amsgrad=True) # learning_rate = 0.001
      # optimizer = optim.Adam(net.parameters(),betas=(beta1, beta2), lr=learning_rate,amsgrad=False) # learning_rate = 0.001




      train_losses = []
      num_epochs = 20
      batch_no=0
      net.train()
      
      c1=0.9
      c2=1.1

      for epoch in range(num_epochs):  # loop over the dataset multiple times
          start_time = time.time()
          running_loss = 0.0
          for i, data in enumerate(trainloader, 0):
              net.train()

              batch_no+=1

              # get the inputs; data is a list of [inputs, labels]
              inputs, labels = data[0].to(device), data[1].to(device)

              # zero the parameter gradients
              optimizer.zero_grad()


              if batch_no >=200:
                 for param_group in optimizer.param_groups:
                    param_group['lr'] *=torch.FloatTensor(1).uniform_(c1**(1/(batch_no%100+1)), c2**(1/(batch_no%100+1))).item()
                    # param_group['lr'] *=torch.FloatTensor(1).uniform_(c1**(1/(batch_no)), c2**(1/(batch_no))).item()

              # forward + backward + optimize
              outputs = net(inputs)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()
              
              # print statistics
              running_loss += loss.item()
              if i % 195 ==  194:    # print every 2000 mini-batch
                  print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 194))
                  # print(time.time()-start_time)
                  train_losses.append(running_loss / 194)
                  running_loss = 0.0

     # output the results
      output = ["{}{}".format("loss: ", "%.3f" % i) for i in train_losses]
      output_conc = output
      np.savetxt('SGD_train'+str(10+randseed)+'.txt', output_conc, fmt="%s")
      files.download('SGD_train'+str(10+randseed)+'.txt')